This first bit of code takes care of a few things. First, it gets some of the necessary imports out of the way and sets pandas print options. Then, it reads in the sequence csv file and filters out unnecessary columns, leaving 5 total features:

1.   The position of the packet within its sequence. This is used only to parse the csv file back out into sequences.
2.   The timestamp of the packet
3.   The packet length
4.   The source port
5.   The destination port

In [0]:
import pandas as pd
import numpy as np
import time
import os
import random
import psutil
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

# Read the data and filter out unnecessary columns
data = pd.read_csv("SequenceCSV.csv", delimiter=',', header=0)
#data = data.filter(items=['Position','Time','Source IP','Destination IP','Length','Source Port','Destination Port'])
data = data.filter(items=['Position','Time','Length','Source Port','Destination Port'])
print(data)
#num_sequences = data.Position.value_counts()[0]
#print(num_sequences)

        Position          Time  Length  Source Port  Destination Port
0              0     37.294817      60           79              1024
1              1     37.295017      60         1024                79
2              2     37.295563      60         1024                79
3              3     37.307251      60           79              1024
4              4     37.327150      60           79              1024
...          ...           ...     ...          ...               ...
465100         3  25253.689350      60        15901                23
465101         0  25253.710980      60         8803                23
465102         1  25253.779420      60        15901                23
465103         2  25253.850980      60         9959                23
465104         3  25253.860990      60         8803                23

[465105 rows x 5 columns]


# Feature vector encoding

In [0]:
# Encode a numeric feature
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd

# Encode a categorical feature
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

encode_numeric_zscore(data, "Time")
encode_numeric_zscore(data, "Length")
encode_numeric_zscore(data, "Source Port")
encode_numeric_zscore(data, "Destination Port")

data.dropna(inplace=True,axis=1)
print(data)

        Position      Time    Length  Source Port  Destination Port
0              0 -2.057470 -0.413749    -0.896562         -0.600173
1              1 -2.057470 -0.413749    -0.802388         -0.698951
2              2 -2.057470 -0.413749    -0.802388         -0.698951
3              3 -2.057468 -0.413749    -0.896562         -0.600173
4              4 -2.057465 -0.413749    -0.896562         -0.600173
...          ...       ...       ...          ...               ...
465100         3  1.951252 -0.413749     0.680176         -0.704804
465101         0  1.951256 -0.413749    -0.027174         -0.704804
465102         1  1.951267 -0.413749     0.680176         -0.704804
465103         2  1.951278 -0.413749     0.088027         -0.704804
465104         3  1.951280 -0.413749    -0.027174         -0.704804

[465105 rows x 5 columns]


# Data preparation

In [0]:
# Get the starting points of each sequence within the dataset
points = []
for x in range(0, len(data.index)):
  if(data['Position'][x] == 0):
    points.append(x)

# Get the input and target sequences for the pairs
pairs = []
for x in range(0, len(points)-1):
  count = 0
  input = []
  target = []
  for y in range(points[x], points[x+1]):
    if(count < 3):
      # Get input sequence
      packet = [data['Time'][y], data['Length'][y], data['Source Port'][y], data['Destination Port'][y]]
      #packet = [data['Time'][y], data['Length'][y]]
      input.append(packet)
      count = count + 1
    else:
      # Get target sequence
      packet = [data['Time'][y], data['Length'][y], data['Source Port'][y], data['Destination Port'][y]]
      #packet = [data['Time'][y], data['Length'][y]]
      target.append(packet)
      count = count + 1

  pair = [input, target]
  pairs.append(pair)

print(random.choice(pairs))
print(len(pairs))

[[[1.325582279934195, -0.41374851003069607, -0.6929669550728281, -0.698846004774689], [1.3255827616219964, -0.41374851003069607, -0.8964619072795385, -0.4854034355178569], [1.3260805629131358, -0.41374851003069607, 0.8362348282826909, -0.7048040000967455]], [[1.3261155353551535, -0.41374851003069607, 0.8362348282826909, -0.7048040000967455]]]
47355


# Make the tensors

The final step of data preparation requires us to transform the [input, target] pairs into tensors so that the model can read them. This is done simply with the following two functions:

1.   tensor_from_sequence - converts the token vectors into tensors
2.   tensors_from_pair - calls the previous function for the input and target sequences of a given pair and returns the resulting tensors

In [0]:
# Functions to prepare the data for insertion into the model
def tensor_from_sequence(sequence):  # Create a tensor from a sequence using the dictionary
    return torch.tensor(sequence, dtype=torch.float, device=device)

def tensors_from_pair(pair):  # Get an input and target tensor out of a pair
    input_tensor = tensor_from_sequence(pair[0])
    target_tensor = tensor_from_sequence(pair[1])
    return (input_tensor, target_tensor)
  
input_tensor, target_tensor = tensors_from_pair(pairs[42500])
print("Input Tensor: \n", input_tensor)
print("Target Tensor: \n", target_tensor)

Input Tensor: 
 tensor([[ 1.5554, -0.4137,  0.6802, -0.7048],
        [ 1.5554, -0.4137,  0.7599, -0.7049],
        [ 1.5554, -0.4137,  0.7345, -0.6988]])
Target Tensor: 
 tensor([[ 1.5554, -0.4137, -0.8965,  1.0118]])


The following few segments of code define the two parts of the sequence to sequence (seq2seq) model: the encoder and the decoder. Each of these two parts is a recurrent neural network (RNN), which is a neural network that performs some operation on a sequence of data and uses the output generated by that operation as input for the next step (recurrence). For these RNNs, we use the **Gated Recurrent Unit** (GRU) architecture, as opposed to the more commonly used **Long Short Term Memory** (LSTM) architecture. This is because, despite being a newer architecture, GRU works similarly to LSTM and has been shown to yield similar results while being slightly more efficient computationally. [This paper](https://arxiv.org/pdf/1412.3555v1.pdf) gives a more in-depth overview of the differences between the two architectures.
# Encoder
In a seq2seq model using an encoder and decoder, the responsibility of the encoder is to encode, or condense, the input sequence into a single vector while retaining the original meaning of that sequence. For each packet in the input sequence, the encoder will produce two things using the embedding layers:



1.   A **vector** (called output_vector in the following code)
2.   A **hidden state** (called hidden_state in the following code)



Following this, a new input sequence and the previous hidden state will be taken as input to do the next step on the next packet in the sequence, and the output vector will be adjusted accordingly and a new hidden state produced. This process is repeated until a final output vector (the **context vector**) is reached, which will be given to the decoder later on. The forward function carries out these tasks in our implementation. 

This implementation is a bit different from a typical Encoder RNN, however, because it uses multiple features as input. Since there are multiple features, the size of the GRU must be multiplied by the number of total features, which is shown on line 7.

In [0]:
# Recurrent neural network for Encoder of the seq2seq model
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size) # Applies Gated Recurrent Unit (GRU) to input sequence

    def forward(self, input_token, hidden_state):
        input_token = input_token.unsqueeze(0).unsqueeze(0)
        #print(input_token)
        output_vector, hidden_state = self.gru(input_token, hidden_state)
        return output_vector, hidden_state

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Decoder

The decoder, like the encoder, is a recurrent neural network using GRU architecture. The decoder takes the context vector as its initial hidden state. As before, the forward function carries out the necessary steps, taking an input token and hidden state as input, then producing an output vector and new hidden state. Unlike the encoder, however, the decoder applies the softmax function to the output vector for normalization.

In [0]:
# Recurrent neural network for Decoder of seq2seq model
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(output_size, hidden_size) # Applies GRU
        self.out = nn.Linear(hidden_size, output_size)
        self.relu = nn.LeakyReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_token, hidden_state):
        input_token = input_token.unsqueeze(0).unsqueeze(0)
        output_vector = self.relu(input_token)
        #print(output_vector)
        output_vector, hidden_state = self.gru(output_vector, hidden_state)
        output_vector = self.out(output_vector[0])
        return output_vector, hidden_state

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Helper functions

In [0]:
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Helper functions to keep track of the time elapsed and time remaining
def as_minutes(sec):
    mins = math.floor(sec / 60)
    sec = sec - (mins * 60)
    return '%dm %ds' % (mins, sec)

def time_since(since, percent):
    now = time.time()
    sec = now-since
    es = sec/(percent)
    rs = es-sec
    return '%sec (- %sec)' % (as_minutes(sec), as_minutes(rs))

# Plot loss vs number of iterations
def show_plot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.5) # Put plot ticks at intervals of 0.5
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

# Training function
The training function performs the core aspect of machine learning model optimization. In this implementation, it takes many parameters:

1. Input tensor which has the tensor representation of the first 3 packets in the sequence
2. Target tensor
3. The encoder and decoder that the function is going to be training
4. Optimizers for the encoder and decoder to oversee the machine learning process
5. A function (Negative log likelihood loss) to calculate the training loss, which is called "criterion" in the code
6. The maximum length a sequence can have

The function follows a process of several steps, beginning with preparation, which includes initializing the hidden state of the encoder, wiping the gradients of the optimizers, and getting the lengths of the input and target sequences. Next, we have to run the encoder on all of the tokens in the input sequence, which is handled in a loop. Once the encoder has produced an output vector, it is passed to the decoder along with the decoder's input token – the most recently predicted token if teacher forcing is not being used, and the actual target token if it is. After each prediction, the total loss is increased by comparing the predicted token to the expected target using the loss function. Finally, the optimizers are updated using the results of this iteration of the model, and the average loss is returned.

In [0]:
teacher_forcing_ratio = 0 # Make 0 if don't want teacher forcing
MAX_LENGTH = 320

# Training function
# Criterion = negative log likelihood loss (NLLLoss)
# Lines commented out are for attn decoder
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.init_hidden() # Initialize hidden state of the encoder
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    input_length = input_tensor.size(0) # length of the input sequence
    target_length = target_tensor.size(0) # length of the target sequence
    loss = 0

# loop through the input tokens w/ encoder and get the final vector/hidden state
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

    decoder_input = target_tensor[0]
    decoder_hidden = encoder_hidden # Initialize the hidden state of the decoder
    # Decide whether to use teacher forcing on this run
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Run the decoder for each element of the target sequence
    if use_teacher_forcing:
      for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        loss = loss + criterion(decoder_output, target_tensor[di])
        decoder_input = target_tensor[di]  # Teacher forcing

    else:
      for di in range(target_length):
        #print(decoder_input)
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        decoder_input = decoder_output.squeeze()
        loss = loss + criterion(decoder_output, target_tensor[di].unsqueeze(0))

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length

# Training overhead
In order to train the model using an entire training dataset, an overhead function is required to oversee the process. This function takes the encoder and decoder as parameters, along with a few user-defined training variables including the number of iterations to train over, intervals at which to print and plot results, and the learning rate, which determines how much the optimizer will be influenced by each iteration.

The function defines the optimizers for the encoder and decoder, takes a series of random input, target pairs, defines the criterion function, and runs the train function on a different pair for the number of iterations specified by the user. Periodically, progress is printed using the previously defined helper functions, so that the user can keep track of the model's performance.

In [0]:
# Repeatedly run the train function and print evaluation info as it goes
def train_iterations(encoder, decoder, n_iterations, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    # Get the training pairs
    training_pairs = [tensors_from_pair(random.choice(pairs)) for i in range(n_iterations)]
    criterion = nn.MSELoss()

    # Loop to train the model with the specified number of iterations
    for iteration in range(1, n_iterations + 1):
        training_pair = training_pairs[iteration - 1]
        input_tensor = training_pair[0] # Get an input tensor from the pair
        target_tensor = training_pair[1]  # Get a target tensor from the pair

        # Train the model on the pairs
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        loss_total = loss_total + loss
        plot_loss_total = plot_loss_total + loss

        if(iteration == 40000):
          torch.save(encoder.state_dict(), '/content/encoder20percent.dict')
          torch.save(decoder.state_dict(), '/content/decoder20percent.dict')
        
        if(iteration == 80000):
          torch.save(encoder.state_dict(), '/content/encoder40percent.dict')
          torch.save(decoder.state_dict(), '/content/decoder40percent.dict')

        if(iteration == 110000):
          torch.save(encoder.state_dict(), '/content/encoder55percent.dict')
          torch.save(decoder.state_dict(), '/content/decoder55percent.dict')

        if(iteration == 140000):
          torch.save(encoder.state_dict(), '/content/encoder70percent.dict')
          torch.save(decoder.state_dict(), '/content/decoder70percent.dict')

        if(iteration == 170000):
          torch.save(encoder.state_dict(), '/content/encoder85percent.dict')
          torch.save(decoder.state_dict(), '/content/decoder85percent.dict')

        if(iteration == 200000):
          torch.save(encoder.state_dict(), '/content/encoder100percent.dict')
          torch.save(decoder.state_dict(), '/content/decoder100percent.dict')

        # If it has reached the print interval, print progress information
        if iteration % print_every == 0:
            loss_avg = loss_total/print_every
            loss_total = 0
            print('%s (%d %d%%) %.4f' % (time_since(start, iteration / n_iterations), iteration, iteration / n_iterations * 100, loss_avg))

        # If it has reached the plot interval, add info to the plot_losses array
        if iteration % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    show_plot(plot_losses)

# Train the model

Define a size for the hidden vector, initialize an encoder and decoder, then run the train_iterations function.

NOTE: SKIP IF TESTING A TRAINED MODEL

In [0]:
hidden_size = 256
encoder1 = Encoder(input_size=4,hidden_size=hidden_size).to(device)
decoder1 = Decoder(hidden_size=hidden_size,output_size=4).to(device)

# originally had iterations at 40,000
train_iterations(encoder1, decoder1, n_iterations=200000, print_every=1000, learning_rate=0.0002)